In [2]:
#NEW MODEL

#ROLLING (absolute) MEAN OF NORMALIZED DATA

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import numpy as np
import os

input_dir = "mindrove_dataset"
dataval= pd.read_csv(os.path.join(input_dir, "definitivo_dataset.csv"))

emgval = np.abs(dataval.iloc[:,2:10] * 0.045)
data = np.array(emgval)

#FIRST: NORMALIZE
normalized_data = np.empty_like(data)
scalers = []
for i in range(data.shape[1]):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(data[:, i].reshape(-1, 1))
    scalers.append(scaler)
for i, scaler in enumerate(scalers):
    normalized_data[:, i] = scaler.transform(data[:, i].reshape(-1, 1)).flatten()

taskval = dataval.iloc[:,1]
newdata = np.zeros(((data.shape[0]),9))
newdata[:, 0] = taskval
newdata[:, 1:] = normalized_data

#SECOND: ROLLING MEAN FOR EACH COLUMN
newdata = pd.DataFrame(newdata) #convert to dataframe
roll = newdata.rolling(window=10, step=8).mean()
roll2 = np.zeros(roll.shape)

repetitions = [1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6,7,7,7]
count=-1
#he establecido un treshold para que asigne un task u otro y no haya decimales y
#que en total solo sean 7 tasks

for i in range(1,(len(roll))):
        if not roll.iloc[i,0] == 0 and roll.iloc[i-1,0] == 0:
            count += 1
        if not roll.iloc[i,0] == 0 and roll.iloc[i,0] < (repetitions[count]/2):
            roll2[i,0] = 0
        elif not roll.iloc[i,0] == 0 and roll.iloc[i,0] >= (repetitions[count]/2):
            roll2[i,0] = repetitions[count]
#eliminate first 2 values NaN
roll2[9:,1:]= roll.iloc[9:,1:]
MAV = roll2[9:,:]


In [3]:
#ROLLING SSI OF NORMALIZED DATA

#FIRST: SQUARED THE NORMALIZED DATA
normsquared = np.square(normalized_data)   
taskval = dataval.iloc[:,1]
newdata = np.zeros(((data.shape[0]),9))
newdata[:, 0] = taskval
newdata[:, 1:] = normsquared

#SECOND: ROLLING SSI FOR EACH COLUMN
import pandas as pd
newdata = pd.DataFrame(newdata) #convert to dataframe
roll = newdata.rolling(window=10, step=8).mean()
roll2 = np.zeros(roll.shape)
repetitions = [1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6,7,7,7]
count=-1
#he establecido un treshold para que asigne un task u otro y no haya decimales y
#que en total solo sean 7 tasks

for i in range(1,(len(roll))):
        if not roll.iloc[i,0] == 0 and roll.iloc[i-1,0] == 0:
            count += 1
        if not roll.iloc[i,0] == 0 and roll.iloc[i,0] < (repetitions[count]/2):
            roll2[i,0] = 0
        elif not roll.iloc[i,0] == 0 and roll.iloc[i,0] >= (repetitions[count]/2):
            roll2[i,0] = repetitions[count]
#eliminate first 9 values NaN           
roll2[9:,1:]= roll.iloc[9:,1:]
SSI = roll2[9:,:]

In [4]:
#JOIN
datasvm = np.concatenate([MAV, SSI[:,1:]], axis=1)


#BEST PARAMETERS
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score

# Split data into training and validation sets
X = datasvm[:,1:] #features
y = datasvm[:,0] #labels
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=44)

# Define grid of parameter values for Gamma and C Parameters
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = {'gamma': gamma_range, 'C': C_range}
 
# Define SVM model with RBF kernel
svm = SVC(kernel='rbf')
 
# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)
 
# Print best parameter values and accuracy on validation set
print("Best gamma value: ", grid_search.best_params_['gamma'])
print("Best C value: ", grid_search.best_params_['C'])
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Validation accuracy: ", accuracy)


In [ ]:
#SVM
import pandas as pd
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
X = datasvm[:,1:] #features
y = datasvm[:,0] #labels
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=44)
rbf = svm.SVC(kernel='rbf', gamma=4, C=5).fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

rbf_precision = precision_score(y_test, rbf_pred, average =  'weighted')
rbf_recall = recall_score(y_test, rbf_pred, average ='weighted')
print('Precision (RBF Kernel): ', "%.2f" % (rbf_precision*100))
print('Recall (RBF Kernel): ', "%.2f" % (rbf_recall*100))


Accuracy (RBF Kernel):  60.57
F1 (RBF Kernel):  56.51
Precision (RBF Kernel):  58.18
Recall (RBF Kernel):  60.57
